In [8]:
from __future__ import division
from datetime import timedelta
from datetime import datetime
from time import mktime
from unidecode import unidecode
from sklearn import preprocessing
import pandas as pd
import os
import numpy as np
import matplotlib as mpl 
import time
import matplotlib.pyplot as plt
import seaborn as sns
import arff
import datetime
import time


In [9]:
#initialise all functions
#-------------------------------------------------------------------------


#print df shape function
def print_shape(df1):
    for i in range(len(df1)):
        print(str(i) + ' '+ str(df1[i].shape))
#-------------------------------------------------------------------------        

# visualise how total row count changes to NaN value treshold to drop row.
def visualise_treshold(df1):
    x_ratio = []
    y_filtered = []
    y_ratio = 0.80
    for r in range(70,101,1):
        dfnew = []
        y_rows = 0
        y_ratio = r/100
        for i in range(len(df1)):
            y_tresh = len(df1[i].columns) * y_ratio
            y_rows += df1[i].dropna(axis=0, thresh=y_tresh).shape[0]
        x_ratio.append(y_ratio)
        y_filtered.append(y_rows)

    print(x_ratio)
    print(y_filtered)

    plt.plot(x_ratio,y_filtered)
    plt.ylabel('total rows')
    plt.xlabel('portion of noNAN')
    plt.show()
#-------------------------------------------------------------------------

# function - drop rows with lot of NA values and reindex
def dropNaNrows(df1,y_ratio=0.9):
    for i in range(len(df1)):
        y_tresh = len(df1[i].columns) * y_ratio
        df1[i] = df1[i].dropna(axis=0, thresh=y_tresh)
        df1[i] = df1[i].reset_index(drop=True)
#-------------------------------------------------------------------------

#drop all constants, but not label
def drop_const_column(df1, label='driver'):
    dfAll_no_const = []
    for i in range(len(df1)):
        df_temp = []
#         print(str(i) + str(df1[i].shape))
        df_temp = (df1[i].loc[:, (df1[i] != df1[i].iloc[0]).any()])
        df_temp[label] = df1[i][label]
        dfAll_no_const.append(df_temp)
#         print(str(i) + str(dfAll_no_const[i].shape))
    return dfAll_no_const
#-------------------------------------------------------------------------

#function - drop columns that do not repeat in other dataframes.
def drop_umatching_columns(dfs, label_column = '', drop_column = ''):
    #create dictionarry of data frame columns
    col_dic = []
    for i in range(len(dfs)):
        col_dic.append(dfs[i].columns.values.tolist())
    print('number of files : ' + str(len(col_dic)))
    a = set.intersection(*[set(list1) for list1 in col_dic])
    ab = sorted(list(a))
    if drop_column != '' : ab.remove(drop_column)
    if label_column != '' :
        ab.remove(label_column)
        ab.append(label_column)
#     return ab
    for i in range(len(dfs)):
        dfs[i] = dfs[i][(ab)]
    print('number of same atributes : ' + str(len(ab)))
    return dfs
#-------------------------------------------------------------------------

#function - remove non ascii characters from column names
def remove_non_ascii_column(df_all_la):
    def remove_non_ascii(text):
        return unidecode(unicode(text, encoding = "utf-8"))

    col_2 = []
    col_1 = df_all_la.columns.values.tolist()
    for i in range(len(col_1)):
        a = remove_non_ascii(col_1[i]).replace(' ', '_')
        a = a.replace('%', 'prc')
        col_2.append(a)

#     df_all_new_names = df_all_la
    for i in range(len(df_all_la.columns.values.tolist())):
        df_all_la.rename(columns={col_1[i]:col_2[i]},inplace=True)
    
    df_all_la = df_all_la.reset_index(drop=True)
#     return df_all_la
#-------------------------------------------------------------------------

#convert columns to numeric, 1 df as input
def convert_numeric(df_input, expetion_columns=''):
    for j in range(len(df_input.columns.values.tolist())-1):
        if df_input.columns[j] == 'Device_Time' or df_input.columns[j] == 'driver':
            continue
        df_input.iloc[:,j] = pd.to_numeric(df_input.iloc[:,j], errors='coerce')
        df_input.fillna(0)
#-------------------------------------------------------------------------

#create csv file 
def write_csv(dframe, name='', pathtofolder='', delimiter='\t'):
    
    #check if path is given
    if pathtofolder == '':
        path_to_results = '../driving data/colt-all-atributes/concat/'
    else:
        path_to_results = pathtofolder
        
    if delimiter=='\t':
        delim = 'tab'
    else:
        delim = str(delimiter)
        
    atributes = str(len(dframe.columns.tolist()))
    #check if name provided
    if name == '' :
        name = atributes + '_atributes'
    
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H%M')
    
    dframe = dframe.fillna(0)
    dframe.to_csv(path_to_results + name +'_'+ delim +'_'+ st + '.csv', sep=delimiter, encoding='utf-8', index=False, na_rep='')
#-------------------------------------------------------------------------
    
    
#write pandas DataFrame to .arff file for WEKA.
#arguments passed: (dataFrame, name='', path_to_folfer='/', label_column='')
# dataFrame - comuplsary. dataframe to export 
# name - optional. Start name of the file appended with tymestamp. if no name only time stamp
# path_to_folder - optional. If no path specified, saves in current folder.
# label_column - optional. Column containing class labels. Takes last if not provided. 
def write_arff(dataFrame, name='', pathtofolder='', label_column=''):
    if pathtofolder == '':
        path_to_results = '../driving data/colt-all-atributes/concat/'
        
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H%M')
    filename = path_to_results + str(name) + '_' + st + '.arff'
    # Open the file with writing permission
    myfile = open(filename, 'w')
    
    #write file heading
    myfile.write('%blabla\n\n')
    myfile.write('@relation obd2\n')
    
    #meke column names and types
    for c in dataFrame.columns:
        if (dataFrame[c].dtype == 'object'):
            wr = '@attribute ' + str(c) +' {' + (','.join(map(str,dataFrame[c].unique()))) + '}'
            myfile.write( wr + '\n')
        else:
            wr = '@attribute ' + str(c) +' numeric'
            myfile.write( wr + '\n')
    

    # Write data to file 
    myfile.write('\n@data\n')
    
    for j in dataFrame.values:
        wr = (','.join(map(str,j)))
        myfile.write( wr + '\n')

    # Close the file
    myfile.close()
    
    print(filename + ' created successfuly')
#-------------------------------------------------------------------------
    

#write function to create sliding windows and lag windows
def roll_lag(df, window_size, lags=0, y_round=3, atributes_to_ignore=''):
    
    if df.empty:
        print ('no DF passed to function roll_lag')
        return
    
    if window_size < 2:
        print('window size should be bigger than 1')
        return
    
    #create temp variables
    df_sythetic = pd.DataFrame()
    result_mean = {}
    result_lag = {}
    result_mean.clear()
    result_lag.clear()

    
    for name in df.columns:
        if name in atributes_to_ignore:
            continue
        col_name = str(window_size) + 's_mean_' + name
        result = df[name].rolling(window_size).mean().fillna(0).round(y_round)
        result_mean.update({ col_name : result })
        if lags > 0 :
            for j in range(lags):
                lag_name = (str(j+1) + '_lag_' + name)
                y_lag = (j+1) * (-1)
                lag_result = result.shift(periods=y_lag).fillna(0)
                result_lag.update({ lag_name : lag_result })
                
    for x in result_mean:
        df_sythetic[x] = result_mean[x]
    if len(result_lag) > 0:
        for x in result_lag:
            df_sythetic[x] = result_lag[x]
    
    df_new = pd.concat([df_sythetic, df], axis=1, sort=False)
    
    #sort columns in df
    col_list = df_new.columns.values.tolist()
    col_list.remove('driver')
    col_list = sorted(col_list)
    col_list.append('driver')
    df_new = df_new[(col_list)]
    return df_new
#-------------------------------------------------------------------------

# normalize data function. 
def prepr_norm(df, ignore='', l = 'l2'):
    df_temp = pd.DataFrame()
    col = df.columns.values.tolist()
    
    if ignore:
        for r in ignore:
            col.remove(r)

    x = df[col].values
    normalized_X = preprocessing.normalize(x,norm=l)
    df_temp = pd.DataFrame(normalized_X, columns=col, index = df.index)

    if ignore:
        for r in ignore:    
            df_temp[r] = df[r]
            
    return df_temp
#-------------------------------------------------------------------------